<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Описание данных**

**Признаки**

RowNumber — индекс строки в данных

CustomerId — уникальный идентификатор клиента

Surname — фамилия

CreditScore — кредитный рейтинг

Geography — страна проживания

Gender — пол

Age — возраст

Tenure — сколько лет человек является клиентом банка

Balance — баланс на счёте

NumOfProducts — количество продуктов банка, используемых клиентом

HasCrCard — наличие кредитной карты

IsActiveMember — активность клиента

EstimatedSalary — предполагаемая зарплата

**Целевой признак**

Exited — факт ухода клиента

## Подготовка данных

In [1]:
!pip install scikit-learn==1.1.3

     |████████████████████████████████| 30.8 MB 892 kB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [2]:
!pip install imblearn

     |████████████████████████████████| 226 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 20.3 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [3]:
import pandas as pd
import warnings 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score, recall_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

In [4]:
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv('/datasets/Churn.csv')

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Для продолжения анализа можно исключить определенные столбцы, а именно RowNumber, CustomerId и Surname. Они могут быть удалены, поскольку они не оказывают влияния на модель.

In [8]:
data = data.drop(["RowNumber","CustomerId","Surname"], axis = 1)

Проверяем наличие повторяющихся значений и пропущенных данных в столбцах.

In [9]:
data.duplicated().sum()

0

In [10]:
data.isna().sum()

CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Заметно, что в столбце Tenure имеются пропущенные значения. Чтобы избежать возможных искажений данных, эти значения можно удалить, поскольку их процент от общего числа значений незначительный.

In [11]:
data = data.dropna(subset = ["Tenure"], axis = 0)

Мы будем подготавливать данные с помощью метода ОНЕ.

In [13]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Вывод: Таким образом, данные готовы для последующего анализа и обработки.

## Исследование задачи

Выберем целевой признак: Exited — факт ухода клиента

Теперь необходимо произвести разбиение выборки на три части: обучающую, валидационную и тестовую.

In [14]:
train, test = train_test_split(data, test_size=0.4, random_state=12345)
features = test.drop('Exited', axis=1)
target = test['Exited']

In [15]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
features_train = train.drop('Exited', axis=1)
target_train = train['Exited']

**Корректировка по замечаниям**

Применим метод OneHotEncoder к категориальным признакам.

In [16]:
categorial_signs = ['Geography', 'Gender']

In [17]:
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [18]:
encoder_ohe.fit(features_train[categorial_signs])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [19]:
features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[categorial_signs])
features_train = features_train.drop(categorial_signs, axis=1)
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9344,727,28,2.0,110997.76,1,1,0,101433.76,0.0,0.0,0.0
3796,537,26,7.0,106397.75,1,0,0,103563.23,0.0,0.0,1.0
7462,610,40,9.0,0.00,1,1,1,149602.54,0.0,0.0,1.0
1508,576,36,6.0,0.00,2,1,1,48314.00,0.0,0.0,1.0
4478,549,31,4.0,0.00,2,0,1,25684.85,0.0,0.0,1.0


In [20]:
features_valid[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid[categorial_signs])
features_valid = features_valid.drop(categorial_signs, axis=1)
features_valid.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7445,516,45,4.0,0.00,1,1,0,95273.73,0.0,0.0,0.0
8620,768,40,8.0,0.00,2,0,1,69080.46,0.0,0.0,0.0
1714,730,45,6.0,152880.97,1,0,0,162478.11,1.0,0.0,1.0
5441,751,29,1.0,135536.50,1,1,0,66825.33,0.0,0.0,1.0
9001,688,32,6.0,124179.30,1,1,1,138759.15,0.0,1.0,1.0


In [21]:
features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[categorial_signs])
features_test = features_test.drop(categorial_signs, axis=1)
features_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
5170,814,31,4.0,0.00,2,1,1,142029.17,0.0,0.0,0.0
4180,607,36,10.0,106702.94,2,0,0,198313.69,1.0,0.0,1.0
7349,632,42,6.0,59972.26,2,0,1,148172.94,1.0,0.0,1.0
7469,686,35,8.0,105419.73,1,1,0,35356.46,0.0,0.0,0.0
3467,538,42,1.0,98548.62,2,0,1,94047.75,1.0,0.0,0.0


Произведем масштабирование данных (это необходимо для применения логистической регрессии):

In [22]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

Теперь проведем масштабирование трех выборок:

In [23]:
features_train[numeric] = scaler.transform(features_train[numeric])

In [24]:
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [25]:
features_test[numeric] = scaler.transform(features_test[numeric])

Проверим, сбалансированы ли классы.

In [26]:
target_train.value_counts(normalize = 1)

0    0.793546
1    0.206454
Name: Exited, dtype: float64

In [27]:
target_valid.value_counts(normalize = 1)

0    0.792629
1    0.207371
Name: Exited, dtype: float64

Вывод: Из результата следует, что присутствует дисбаланс классов: ответов с меткой 0 составляют 80%, а ответов с меткой 1 - 20%.

# Изучим три модели.

 Давайте рассмотрим модель "Решающее дерево".

In [28]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_train , target_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(predictions, target_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 7 F1: 0.580952380952381 AUC-ROC 0.6797786314764468
CPU times: user 673 ms, sys: 343 µs, total: 673 ms
Wall time: 677 ms


 Давайте рассмотрим модель "Случайный лес"

In [29]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_train , target_train)
            predictions = model.predict(features_valid)
            f1 = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 16 Количество деревьев: 15 F1: 0.6026936026936026 AUC-ROC 0.8650933167911321 4
CPU times: user 1min 9s, sys: 184 ms, total: 1min 9s
Wall time: 1min 9s


 Давайте рассмотрим модель"Логическая регрессия"

In [30]:
%%time

model = LogisticRegression()
model.fit(features_train , target_train)
predictions = model.predict(features_valid)
f1 = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.30400000000000005 AUC-ROC 0.773663293800172
CPU times: user 219 ms, sys: 325 ms, total: 544 ms
Wall time: 545 ms


Из данных можно сделать следующие выводы: наиболее точной моделью согласно метрике точности является "Случайный лес", однако показатель F1 у всех моделей невысокий, что указывает на низкое качество моделей.

## Борьба с дисбалансом

1 способ борьбы - upsampling, то есть увеличение выборки

In [33]:
upsample = SMOTE(random_state=42)

In [34]:
features_upsampled_train, target_upsampled_train = upsample.fit_resample(features_train, target_train)

In [35]:
target_upsampled_train.value_counts(normalize = 1)

0    0.5
1    0.5
Name: Exited, dtype: float64

2 способ борьбы - downsampling, то есть уменьшение выборки

In [36]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [37]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.25)

In [38]:
target_downsampled_train.value_counts(normalize = 1)

1    0.509964
0    0.490036
Name: Exited, dtype: float64

Проверим работу наших моделей на данных, которые мы сбалансировали методом upsample

Дерево решений

In [39]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_upsampled_train, target_upsampled_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(predictions, target_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 8 F1: 0.5807228915662652 AUC-ROC 0.7083166898907884
CPU times: user 1.17 s, sys: 0 ns, total: 1.17 s
Wall time: 1.18 s


Случаный лес

In [40]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_upsampled_train, target_upsampled_train)
            predictions = model.predict(features_valid)
            f1 = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 9 Количество деревьев: 30 F1: 0.6509900990099009 AUC-ROC 0.8614283847239889 4
CPU times: user 1min 50s, sys: 290 ms, total: 1min 50s
Wall time: 1min 51s


Линейная регрессия

In [41]:
%%time

model = LogisticRegression()
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_valid)
f1 = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.49375600384245916 AUC-ROC 0.771439668517847
CPU times: user 168 ms, sys: 397 ms, total: 565 ms
Wall time: 545 ms


Проверим работу наших моделей на данных, которые мы сбалансировали методом downsample.

Дерево решений

In [42]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_downsampled_train, target_downsampled_train)
    predictions = model.predict(features_valid)
    f1 = f1_score(predictions, target_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 6 F1: 0.5636704119850188 AUC-ROC 0.7048247882677995
CPU times: user 366 ms, sys: 59.9 ms, total: 426 ms
Wall time: 491 ms


Случайный лес

In [43]:
%%time

f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_downsampled_train, target_downsampled_train)
            predictions = model.predict(features_valid)
            f1 = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 16 Количество деревьев: 20 F1: 0.6035502958579881 AUC-ROC 0.8628917805016778 4
CPU times: user 42.9 s, sys: 175 ms, total: 43 s
Wall time: 43.1 s


Линейная регрессия

In [44]:
%%time

model = LogisticRegression()
model.fit(features_downsampled_train, target_downsampled_train)
predictions = model.predict(features_valid)
f1 = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.5044883303411131 AUC-ROC 0.7770245022153419
CPU times: user 267 ms, sys: 541 ms, total: 808 ms
Wall time: 768 ms


Благодаря проведенной балансировке классов мы достигли показателя F1 меры, превышающего 0,59. Для тестовой выборки мы выберем модель "Случайный лес", так как она показала наилучший результат F1 меры. Для данной модели следует выбирать определенные гиперпараметры: max_depth = 18, n_estimators = 45 и min_samples_leaf = 4.

## Тестирование модели

In [45]:
%%time

model = RandomForestClassifier(max_depth=18, n_estimators=45, min_samples_leaf=4, random_state=1234)
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_test)
f1 = f1_score(predictions, target_test)
probabilities_one_valid = model.predict_proba(features_test)[:, 1]
AUC = roc_auc_score(target_test, probabilities_one_valid)

print("F1:",f1,'AUC-ROC',AUC)

F1: 0.6000000000000001 AUC-ROC 0.8521390810219148
CPU times: user 569 ms, sys: 40.1 ms, total: 609 ms
Wall time: 623 ms


In [47]:
recall = recall_score(model.predict(features_test), target_test)
print(f'Recall = {recall}')

Recall = 0.5778364116094987


Recall модели составил 0.577, что позволило выявить 57,7% потенциально уходящих клиентов.

F1 мера 0.598

Из проведенного сравнения трех моделей (дерево решений, случайный лес и линейная регрессия) лучший результат показала модель "Случайный лес" с определенными гиперпараметрами: max_depth = 18, n_estimators = 45, min_samples_leaf = 4 и random_state = 1234. Это дало нам результат меры F1 на тестовой выборке, который превышает 0,59.